### load data

In [1]:
import pandas as pd

In [2]:
# load data
inspection = pd.read_csv('final_inspection.csv')
reviews = pd.read_csv('review_result_final.csv')
yelp_link = pd.read_csv('final_business_yelplink.csv')

In [120]:
len(yelp_link)

834

In [ ]:
# total number 

In [3]:
original_keywords = [['poison', 'salmonella', 'toxin'],
['dirty', 'dusty', 'filthy', 'greasy', 'messy', 'polluted', 'nasty', 'sloppy'],
['good', 'great', 'nice', 'excellent', 'wonderful', 'positive','amazing'],
['clean', 'fresh', 'washed', 'shining', 'sanitary', 'healthy'],
['fancy', 'elegant', 'rich'],
['vegan', 'vegetarian'],
['sick', 'nauseous', 'suffer'],
['hazardous', 'dangerous', 'risky', 'difficult'],
['improper', 'inappropriate', 'wrong', 'inadmissible'],
['hate', 'horrible', 'trouble', 'painful'],
['dick', 'ass', 'asshole'],
['delicious', 'tasty', 'yummy', 'savory', 'tempting'],
['bug', 'fly', 'germ', 'virus', 'bacteria'],
['broken', 'damaged', 'crushed', 'ruptured', 'collapsed'],
['violate', 'breach', 'offend']]

In [4]:
keywords = ['poison', 'dirty', 'good', 'clean', 'fancy', 
            'vegan', 'sick', 'hazardous', 'improper', 
            'hate', 'dick', 'delicious', 'bug', 'broken', 'violate']

In [104]:
column_names = ['business_id', 'neighborhood', 'stars', 'dollar_sign', 'Category Name',
                'Inspection Demerits', 'Inspection Grade','inspection_year', 'Date', 'senti_neg', 'senti_neu', 
                'senti_pos', 'senti_comp', 'counts', 'weighted_counts', 'reviewer_rating']

In [105]:
for i in range(len(keywords)):
    temp = 'key_' + keywords[i]
    temp1 = 'w_key_' + keywords[i]
    column_names.append(temp)
    column_names.append(temp1)

In [106]:
column_names

['business_id',
 'neighborhood',
 'stars',
 'dollar_sign',
 'Category Name',
 'Inspection Demerits',
 'Inspection Grade',
 'inspection_year',
 'Date',
 'senti_neg',
 'senti_neu',
 'senti_pos',
 'senti_comp',
 'counts',
 'weighted_counts',
 'reviewer_rating',
 'key_poison',
 'w_key_poison',
 'key_dirty',
 'w_key_dirty',
 'key_good',
 'w_key_good',
 'key_clean',
 'w_key_clean',
 'key_fancy',
 'w_key_fancy',
 'key_vegan',
 'w_key_vegan',
 'key_sick',
 'w_key_sick',
 'key_hazardous',
 'w_key_hazardous',
 'key_improper',
 'w_key_improper',
 'key_hate',
 'w_key_hate',
 'key_dick',
 'w_key_dick',
 'key_delicious',
 'w_key_delicious',
 'key_bug',
 'w_key_bug',
 'key_broken',
 'w_key_broken',
 'key_violate',
 'w_key_violate']

In [146]:
# empty dataframe for final result
final_data = pd.DataFrame(index=range(len(yelp_link)), columns=column_names)

### join yelp link and review result tables

In [28]:
link_col = 'Unnamed: 0.1.1'

In [98]:
def add(a, b):
    result = [0 for x in range(len(a))]
    for i in range(len(a)):
        if '.' in b[i]:
            result[i] = float(a[i]) + float(b[i])
        else:
            result[i] = int(a[i]) + int(b[i])
    return result

In [47]:
import numpy as np

In [147]:
dropped = [] # index for those don't match yelp links

for i in range(len(yelp_link)):
# for i in range(10):
    # copy past info from yelp_link to final data
    final_data.loc[i, 'business_id'] = yelp_link.loc[i, 'business_id']
    final_data.loc[i, 'neighborhood'] = yelp_link.loc[i, 'neighborhood']
    final_data.loc[i, 'stars'] = yelp_link.loc[i, 'stars']
    final_data.loc[i, 'dollar_sign'] = yelp_link.loc[i, 'dollar_sign']
        
    target_link = yelp_link.loc[i, 'yelp_link']
    
    # temp variable to store data for single review
    temp_num = 0
    temp_counts = [0 for i in range(len(keywords))]
    temp_weighted_counts = [0 for i in range(len(keywords))]
    temp_rating = 0
    temp_senti_comp = 0
    temp_senti_all = [0 for i in range(3)]
    
    # match two tables according to yelp link
    for j in range(len(reviews)):
        
        if reviews.loc[j, link_col] == target_link:
            temp_num += 1
            temp2 = reviews.loc[j, 'counts'][1:-1].split(',')
            temp3 = reviews.loc[j, 'weighted_counts'][1:-1].split(',')
            temp4 = reviews.loc[j, 'senti_all'][1:-1].split(',')
            
            temp_counts = add(temp_counts, temp2)
            temp_weighted_counts = add(temp_weighted_counts, temp3)
            temp_senti_all = add(temp_senti_all, temp4)
            temp_senti_comp += float(reviews.loc[j, 'senti_comp'][1:-1])
            temp_rating += int(reviews.loc[j, 'Rating'])
        if temp_num >= 40:
            break
    
    if temp_num != 0:
        # combine the temp results for one restaurant
        final_data.loc[i, 'review_rating'] = float(temp_rating/temp_num)
        final_data.loc[i, 'senti_comp'] = float(temp_senti_comp/temp_num)
        
        # copy paste the senti individual values
        temp6= list(np.array(temp_senti_all)/temp_num)
        final_data.loc[i, 'senti_neg'] = temp6[0]
        final_data.loc[i, 'senti_neu'] = temp6[1]
        final_data.loc[i, 'senti_pos'] = temp6[2]


        # split the counts lists into individual columns
        temp1 = np.array(temp_counts)/temp_num
        temp1_w = list(np.array(temp_weighted_counts)/temp_num) # weighted counts

        for k in range(len(keywords)):
            key = 'key_' + keywords[k]
            key_w = 'w_key_' + keywords[k]
            final_data.loc[i, key] = temp1[k] 
            final_data.loc[i, key_w] = temp1_w[k] # weighted counts
    else:
        dropped.append(i)
           

In [149]:
final_data

,business_id,neighborhood,stars,dollar_sign,Category Name,Inspection Demerits,Inspection Grade,inspection_year,Date,senti_neg,...,w_key_dick,key_delicious,w_key_delicious,key_bug,w_key_bug,key_broken,w_key_broken,key_violate,w_key_violate,review_rating
0,pUefPUl0plnvk90r_00DOw,Eastside,3.5,NaN,NaN,NaN,NaN,NaN,NaN,0.0555385,...,0,0.230769,1.30769,0.0769231,0.153846,0,0,0.0769231,0.153846,3.076923
1,W-3Sy3fy85mQdd0ZNFKIiw,Chinatown,2.5,2,NaN,NaN,NaN,NaN,NaN,0.07465,...,0.05,0.2,0.35,0.025,0.075,0,0,0,0,2.300000
2,gSR9bmBa0HacSuMX5BIAxg,Westside,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,lv5Bnqlt93NnvNJ2oNcQ8Q,Spring Valley,4,2,NaN,NaN,NaN,NaN,NaN,0.033925,...,0,0.65,5.375,0,0,0,0,0,0,3.800000
4,pRmb8_tAP5y4YPt_viL07g,Westside,4,3,NaN,NaN,NaN,NaN,NaN,0.04145,...,0.1,0.3,1.05,0,0,0.05,0.1,0,0,3.450000
5,uUlq6chIVb0I8AehlET5bQ,NaN,2.5,1,NaN,NaN,NaN,NaN,NaN,0.06235,...,0.05,0.05,0.05,0,0,0,0,0,0,2.650000
6,cPmThZ7Ugpd8eJnIObOy0Q,Summerlin,3,2,NaN,NaN,NaN,NaN,NaN,0.03315,...,0,0.4,1.35,0,0,0,0,0,0,3.750000
7,GQCmcU4aD6GEq7Eo7pKqUg,Southeast,3,1,NaN,NaN,NaN,NaN,NaN,0.0659,...,0.15,0.25,0.35,0,0,0,0,0,0,2.850000
8,iSC_NslJHJTtwoaKn6kggw,Spring Valley,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,mL9sLXHeA0q7aj8-BXXq4g,Sunrise,2.5,1,NaN,NaN,NaN,NaN,NaN,0.147714,...,0,0,0,0,0,0.142857,0.142857,0,0,2.428571


In [150]:
# delete the rows that don't match
final_data.drop(dropped)

,business_id,neighborhood,stars,dollar_sign,Category Name,Inspection Demerits,Inspection Grade,inspection_year,Date,senti_neg,...,w_key_dick,key_delicious,w_key_delicious,key_bug,w_key_bug,key_broken,w_key_broken,key_violate,w_key_violate,review_rating
0,pUefPUl0plnvk90r_00DOw,Eastside,3.5,NaN,NaN,NaN,NaN,NaN,NaN,0.0555385,...,0,0.230769,1.30769,0.0769231,0.153846,0,0,0.0769231,0.153846,3.076923
1,W-3Sy3fy85mQdd0ZNFKIiw,Chinatown,2.5,2,NaN,NaN,NaN,NaN,NaN,0.07465,...,0.05,0.2,0.35,0.025,0.075,0,0,0,0,2.300000
3,lv5Bnqlt93NnvNJ2oNcQ8Q,Spring Valley,4,2,NaN,NaN,NaN,NaN,NaN,0.033925,...,0,0.65,5.375,0,0,0,0,0,0,3.800000
4,pRmb8_tAP5y4YPt_viL07g,Westside,4,3,NaN,NaN,NaN,NaN,NaN,0.04145,...,0.1,0.3,1.05,0,0,0.05,0.1,0,0,3.450000
5,uUlq6chIVb0I8AehlET5bQ,NaN,2.5,1,NaN,NaN,NaN,NaN,NaN,0.06235,...,0.05,0.05,0.05,0,0,0,0,0,0,2.650000
6,cPmThZ7Ugpd8eJnIObOy0Q,Summerlin,3,2,NaN,NaN,NaN,NaN,NaN,0.03315,...,0,0.4,1.35,0,0,0,0,0,0,3.750000
7,GQCmcU4aD6GEq7Eo7pKqUg,Southeast,3,1,NaN,NaN,NaN,NaN,NaN,0.0659,...,0.15,0.25,0.35,0,0,0,0,0,0,2.850000
9,mL9sLXHeA0q7aj8-BXXq4g,Sunrise,2.5,1,NaN,NaN,NaN,NaN,NaN,0.147714,...,0,0,0,0,0,0.142857,0.142857,0,0,2.428571
10,gxAePV_hN-8VGcJAUp8BJw,Eastside,3,2,NaN,NaN,NaN,NaN,NaN,0.0506667,...,0,0,0,0,0,0,0,0,0,3.000000
11,mrnUt3IdLRf4j8WeeGGbpw,University,3,1,NaN,NaN,NaN,NaN,NaN,0.0705,...,0.1,0.15,0.15,0,0,0,0,0,0,2.450000


### match inspection data

In [152]:
inspection.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Serial Number',
       'Permit Number', 'Restaurant Name', 'Location Name', 'Category Name',
       'Address', 'City', 'State', 'Zip', 'Current Demerits', 'Current Grade',
       'Date Current', 'Inspection Date', 'Inspection Time', 'Employee ID',
       'Inspection Type', 'Inspection Demerits', 'Inspection Grade',
       'Permit Status', 'Inspection Result', 'Violations', 'Record Updated',
       'Location 1', 'geometry', 'Nearest_yelp_business', 'Nearest_yelp_name',
       'Nearest_yelp_address', 'Nearest_yelp_id', 'inspection_year'],
      dtype='object')

In [157]:
for i in range(len(final_data)):
# for i in range(2):
    temp_num = 0
    temp_count = 0
    
    for j in range(len(inspection)):
        if inspection.loc[j, 'Nearest_yelp_id'] == final_data.loc[i, 'business_id']:
            temp_count += 1
            
            # copy past info from inspection to final data
            if temp_count == 1:
                final_data.loc[i, 'Category Name'] = inspection.loc[j, 'Category Name']
                ###### don't need those two for midterm report
#                 final_data.loc[i, 'inspection_year'] = inspection.loc[j, 'inspection_year']
               # final_data.loc[i, 'Inspection Grade'] = inspection.loc[j, 'Inspection Grade']
            
            temp_num += int(inspection.loc[j, 'Inspection Demerits'])

    final_data.loc[i, 'Inspection Demerits'] = temp_num/temp_count
            



In [158]:
final_data

,business_id,neighborhood,stars,dollar_sign,Category Name,Inspection Demerits,Inspection Grade,inspection_year,Date,senti_neg,...,w_key_dick,key_delicious,w_key_delicious,key_bug,w_key_bug,key_broken,w_key_broken,key_violate,w_key_violate,review_rating
0,pUefPUl0plnvk90r_00DOw,Eastside,3.5,NaN,Special Kitchen,3,NaN,NaN,NaN,0.0555385,...,0,0.230769,1.30769,0.0769231,0.153846,0,0,0.0769231,0.153846,3.076923
1,W-3Sy3fy85mQdd0ZNFKIiw,Chinatown,2.5,2,Restaurant,5.125,NaN,NaN,NaN,0.07465,...,0.05,0.2,0.35,0.025,0.075,0,0,0,0,2.300000
2,gSR9bmBa0HacSuMX5BIAxg,Westside,3,NaN,Restaurant,6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,lv5Bnqlt93NnvNJ2oNcQ8Q,Spring Valley,4,2,Restaurant,3.8,NaN,NaN,NaN,0.033925,...,0,0.65,5.375,0,0,0,0,0,0,3.800000
4,pRmb8_tAP5y4YPt_viL07g,Westside,4,3,Special Kitchen,4.96552,NaN,NaN,NaN,0.04145,...,0.1,0.3,1.05,0,0,0.05,0.1,0,0,3.450000
5,uUlq6chIVb0I8AehlET5bQ,NaN,2.5,1,Restaurant,6.36364,NaN,NaN,NaN,0.06235,...,0.05,0.05,0.05,0,0,0,0,0,0,2.650000
6,cPmThZ7Ugpd8eJnIObOy0Q,Summerlin,3,2,Restaurant,5.35294,NaN,NaN,NaN,0.03315,...,0,0.4,1.35,0,0,0,0,0,0,3.750000
7,GQCmcU4aD6GEq7Eo7pKqUg,Southeast,3,1,Restaurant,4.58333,NaN,NaN,NaN,0.0659,...,0.15,0.25,0.35,0,0,0,0,0,0,2.850000
8,iSC_NslJHJTtwoaKn6kggw,Spring Valley,4.5,NaN,Restaurant,4.71429,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,mL9sLXHeA0q7aj8-BXXq4g,Sunrise,2.5,1,Restaurant,8.26667,NaN,NaN,NaN,0.147714,...,0,0,0,0,0,0.142857,0.142857,0,0,2.428571
